<a href="https://colab.research.google.com/github/Raph-09/Heroku-Demo/blob/master/webscrabing_salary_com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import requests
from bs4 import BeautifulSoup as bs
import csv
import json
from time import sleep

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'REPLACE_WITH_YOUR_FILE_ID'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [39]:
from google.colab import files

with open('salary.com.ipynb', 'w') as f:
  f.write('ssalary.com.ipynb')

files.download('salary.com.ipynb')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**TUTOR**: **ISREAL DRYER** YOUTUBE

In [ ]:
temp = 'https://www.salary.com/research/salary/alternate/{}-salary/{}'
position = 'senior-accountant'
city = 'charlotte-nc'
url = temp.format(position,city)

In [ ]:

response = requests.get(url)

In [ ]:
response.reason

'OK'

In [ ]:
soup = bs(response.text,"html.parser")


In [ ]:
pattern  = re.compile(r'Occupation')
scripts = soup.find('script',{'type':"application/ld+json"},text = pattern )

In [ ]:
scripts

<script type="application/ld+json">
 {
    "@context": "http://schema.org",
    "@type": "Occupation",
    "name": "Senior Accountant",
    "mainEntityOfPage": {
        "@type": "WebPage",
        "lastReviewed": "2021-02-26T00:00:00Z"
    },
    "description": "The Senior Accountant ensures the accuracy of entries to ledger accounts and reconciles subsidiary ledger accounts to the general ledger. Maintains financial records and ensures that financial transactions are properly recorded. Being a Senior Accountant analyzes current costs, revenues, financial commitments, and obligations incurred to predict future revenues and expenses. Prepares complex balance sheets, profit and loss statements and other financial reports. In addition, Senior Accountant may supervise and guide lower-level accountants. Requires a bachelor&#39;s degree. May require CPA certification. Typically reports to a supervisor or manager. Being a Senior Accountant contributes to moderately complex aspects of a proje

In [ ]:
json_raw = scripts.contents[0]

In [ ]:
json_data =  json.loads(json_raw)#convert to python dictionary

In [ ]:
json_data

{'@context': 'http://schema.org',
 '@type': 'Occupation',
 'description': 'The Senior Accountant ensures the accuracy of entries to ledger accounts and reconciles subsidiary ledger accounts to the general ledger. Maintains financial records and ensures that financial transactions are properly recorded. Being a Senior Accountant analyzes current costs, revenues, financial commitments, and obligations incurred to predict future revenues and expenses. Prepares complex balance sheets, profit and loss statements and other financial reports. In addition, Senior Accountant may supervise and guide lower-level accountants. Requires a bachelor&#39;s degree. May require CPA certification. Typically reports to a supervisor or manager. Being a Senior Accountant contributes to moderately complex aspects of a project. Work is generally independent and collaborative in nature. Working as a Senior Accountant typically requires 4 to 7 years of related experience.',
 'estimatedSalary': [{'@type': 'Moneta

In [ ]:
job_title = json_data['name']
location = json_data['occupationLocation'][0]['name']
description = json_data['description']
ntile_10 = json_data['estimatedSalary'][0]['percentile10']
ntile_25 = json_data['estimatedSalary'][0]['percentile25']
ntile_50 = json_data['estimatedSalary'][0]['median']
ntile_75 = json_data['estimatedSalary'][0]['percentile75']
ntile_90 = json_data['estimatedSalary'][0]['percentile90']

In [ ]:
def extract_salary(job_title,city):
  # extract the salary for a profession in a given city
  temp = 'https://www.salary.com/research/salary/alternate/{}-salary/{}'
  url = temp.format(job_title,city)
  try:
    response = requests.get(url)
    if response.status_code!=200:
      return None
  except requests.exceptions.connection.Error:
    return None

  soup = bs(response.text,"html.parser")
  pattern  = re.compile(r'Occupation')
  scripts = soup.find('script',{'type':"application/ld+json"},text = pattern )
  json_raw = scripts.contents[0]
  json_data =  json.loads(json_raw)

  job_title = json_data['name']
  location = json_data['occupationLocation'][0]['name']
  description = json_data['description']
  ntile_10 = json_data['estimatedSalary'][0]['percentile10']
  ntile_25 = json_data['estimatedSalary'][0]['percentile25']
  median = json_data['estimatedSalary'][0]['median']
  ntile_75 = json_data['estimatedSalary'][0]['percentile75']
  ntile_90 = json_data['estimatedSalary'][0]['percentile90']
  
  salary = (job_title,location,description,ntile_10,ntile_25,median,ntile_75,ntile_90)
  return salary

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
largest_city_path = "/content/drive/MyDrive/datascience_file/largest_uscities.csv"

In [ ]:
with open("/content/drive/MyDrive/datascience_file/largest_uscities.csv",encoding='cp1252', newline='') as f:
   reader = csv.reader(f) 
   cities = [city for row in reader for city in row]

In [ ]:
print(cities[:10])

['New-York-NY', 'Los-Angeles-CA', 'Chicago-IL', 'Houston-TX', 'Phoenix-AZ', 'Philadelphia-PA', 'San-Antonio-TX', 'San-Diego-CA', 'Dallas-TX', 'San-Jose-CA']


In [ ]:
salary_data = []
for city in cities:
  result = extract_salary('senior-accountant',city)
  if result:
    salary_data.append(result)
    sleep(0.5)


In [ ]:
for row in salary_data[:5]:
  print(row)

('Senior Accountant', 'New York, NY', 'The Senior Accountant ensures the accuracy of entries to ledger accounts and reconciles subsidiary ledger accounts to the general ledger. Maintains financial records and ensures that financial transactions are properly recorded. Being a Senior Accountant analyzes current costs, revenues, financial commitments, and obligations incurred to predict future revenues and expenses. Prepares complex balance sheets, profit and loss statements and other financial reports. In addition, Senior Accountant may supervise and guide lower-level accountants. Requires a bachelor&#39;s degree. May require CPA certification. Typically reports to a supervisor or manager. Being a Senior Accountant contributes to moderately complex aspects of a project. Work is generally independent and collaborative in nature. Working as a Senior Accountant typically requires 4 to 7 years of related experience.', '80114', '88490', '97690', '108390', '118132')
('Senior Accountant', 'Los 

In [ ]:
with open('salary_data.csv','w',newline='',encoding='cp1252') as f:
  writer = csv.writer(f)
  writer.writerow(['job_title','location','description','ntile_10','ntile_25','median','ntile_75','ntile_90'])
  writer.writerows(salary_data)
